#Импорт библиотек

In [1]:
import requests
import pandas as pd
import numpy as np
import copy

!pip install xlsxwriter
import xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 2.9 MB/s eta 0:00:00


#Сбор данных

Напишем функцию для получения списка специальностей c источником финансирования - finance 'Бюджет'.

In [2]:
def get_speciality(url):
    user_agent = {'user-agent': 'Mozilla/5.0 (X11; Linux aarch64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.188 Safari/537.36 CrKey/1.54.250320'}
    data = {
        'admissionCampaignType' : "Прием на обучение на бакалавриат/специалитет",
        'financingSource' : 'Бюджет',
        'studyForm' : "Очная"
    }

    raw = requests.post(url=url, headers = user_agent, params = data).json()
    return raw['data']


Напишем функцию, которая будет собирать данные о поступающих на определенное направление - department.

In [3]:
def get_data(url, department, orig):
    # Словарь ключей для получения кол-во мест
    place = {'Особая квота':            'SpecialQuotaCount',
             'Основные места':          'BudgetQuotaCount',
             'Отдельная квота':         'SeparateQuotaCount',
             'Целевая квота':           'TargetQuotaCount',
             'Полное возмещение затрат':'ExtraBudgetQuotaCount'}

    user_agent = {'user-agent': 'Mozilla/5.0 (X11; Linux aarch64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.188 Safari/537.36 CrKey/1.54.250320'}
    data = {
        'admissionCampaignType' : "Прием на обучение на бакалавриат/специалитет",
        'financingSource' : "Бюджет",
        'studyForm' : "Очная",
        'trainingDirection' : department,
        'sortDirection' : "sum",
    }

    raw = requests.post(url=url, headers = user_agent, params = data).json()

    for student in raw.get("data"):
        if student['Code'] == '':
            continue

        if orig == 'Y' and student['AtestOrig'] == 'N':
          continue

        category = student['Category']

        data_students['СНИЛС'].append(student['Code'])
        data_students['Специальность'].append(department)
        data_students['Приоритет'].append(int(student["SelectedPriority"]))
        data_students['БВИ'].append(student["NoExams"] == 'Y')
        data_students['Категория'].append(category)

        try:
            data_students['Балл'].append(int(student['SumScore']))
        except:
            data_students['Балл'].append(np.nan)

        if category not in count_place:
            count_place[category] = {}

        if department not in count_place[category]:
            count_place[category][department] = int(student[place[category]])


In [4]:
# Словарь для хранения информации о кол-во мест
count_place = {}
# Будущий DataFrame для хранения информации об абитуриентах
data_students = {'СНИЛС': [], 'Специальность': [], 'Балл': [], 'Приоритет': [], 'БВИ': [], 'Категория': []}

orig = input("Выберите категорию аттестата: 'N' - не учитывая наличие оригинала, 'Y' - только оригинал документа: ").upper()
assert orig in ["N", "Y"], "Выбрана неверная категория."

Выберите категорию аттестата: 'N' - не учитывая наличие оригинала, 'Y' - только оригинал документа: Y


In [5]:
url_speciality = "https://www.dvfu.ru/bitrix/services/main/ajax.php?admissionCampaignType=%D0%9F%D1%80%D0%B8%D0%B5%D0%BC%20%D0%BD%D0%B0%20%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%BD%D0%B0%20%D0%B1%D0%B0%D0%BA%D0%B0%D0%BB%D0%B0%D0%B2%D1%80%D0%B8%D0%B0%D1%82%2F%D1%81%D0%BF%D0%B5%D1%86%D0%B8%D0%B0%D0%BB%D0%B8%D1%82%D0%B5%D1%82&financingSource=%D0%91%D1%8E%D0%B4%D0%B6%D0%B5%D1%82&studyForm=%D0%9E%D1%87%D0%BD%D0%B0%D1%8F&mode=class&c=dvfu%3Aadmission.spd&action=getTrainingDirectionList"
fefu_speciality = get_speciality(url_speciality)

Формирование pandasDataFrame по всем специальностям.

In [6]:
url_data = "https://www.dvfu.ru/bitrix/services/main/ajax.php?admissionCampaignType=%D0%9F%D1%80%D0%B8%D0%B5%D0%BC%20%D0%BD%D0%B0%20%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%BD%D0%B0%20%D0%B1%D0%B0%D0%BA%D0%B0%D0%BB%D0%B0%D0%B2%D1%80%D0%B8%D0%B0%D1%82%2F%D1%81%D0%BF%D0%B5%D1%86%D0%B8%D0%B0%D0%BB%D0%B8%D1%82%D0%B5%D1%82&financingSource=%D0%91%D1%8E%D0%B4%D0%B6%D0%B5%D1%82&studyForm=%D0%9E%D1%87%D0%BD%D0%B0%D1%8F&trainingDirection=01.03.02%20%D0%9F%D1%80%D0%B8%D0%BA%D0%BB%D0%B0%D0%B4%D0%BD%D0%B0%D1%8F%20%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B0%20%D0%B8%20%D0%B8%D0%BD%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B0&sortDirection=sum&enrolled=N&sendorig=N&topprior=N&mode=class&c=dvfu%3Aadmission.spd&action=getStudents"

for speciality in fefu_speciality:
    try:
        get_data(url_data, speciality, orig)
    except:
        continue

In [7]:
data_students = pd.DataFrame(data_students)
data_students.shape

(3288, 6)

Удалим дубликаты.

In [8]:
data_students = data_students.drop_duplicates()
data_students.shape

(3286, 6)

#Алгоритм

Так как count_place - это словарь словарей, для того чтобы он не изменялся по ходу выполнения программы, сделаем глубокое копирование.

In [9]:
copy_place = copy.deepcopy(count_place)
df_students = data_students.copy()

Создадим пустой DataFrame, который будет хранить информацию о поступивших абитуриентах.

In [10]:
students_received = pd.DataFrame({'СНИЛС': [], 'Специальность': [], 'Балл': [], 'Приоритет': [], 'БВИ': [], 'Категория': []})

Напишем функцию, которая будет считать кол-во оставшихся мест на все специальности в определенной категории мест.

In [11]:
def get_count_place(place, category):
    return sum([place[category][i] for i in place[category]])

Посмотрим на общее кол-во мест в категории: Основные места.

In [12]:
get_count_place(copy_place, 'Основные места')

1891

Напишем функцию, которая будет формировать DataFrame с поступившими абитуриентами по определенной категории (category).

In [13]:
def incoming_students(category):
    global students_received, df_students

    not_students = 0
    while not_students != len(copy_place[category]): # Выполняем цикл до тех пор, пока в
        not_students = 0                              # не останется абитуриентов на все специальности

        for speciality in copy_place[category]:
            # Проверим остались ли места на данную специальность
            if copy_place[category][speciality] == 0:
                not_students += 1
                continue

            # Формируем DataFrame по специальности и сортируем в порядке убывания
            speciality_data = df_students[(df_students['Специальность'] == speciality) & (df_students['Категория'] == category)]\
                              .sort_values(by=['БВИ', 'Балл'], ascending=( False , False ))

            # Делаем выборку из первых абитуриентов
            speciality_data = speciality_data.iloc[:copy_place[category][speciality]]
            # Проверяем не пустой ли speciality_data
            if len(speciality_data) == 0:
                not_students += 1
                continue

            # Перебираем проходящих на данную специальность абитуриентов
            for i, student in speciality_data.iterrows():
                # Проверяем есть ли уже абитуриент в DataFrame, поступивших абитуриентов
                if student['СНИЛС'] in students_received['СНИЛС'].to_list():
                    # Проверяем меньше ли новый приоритет
                    if student['Приоритет'] < students_received[students_received['СНИЛС'] == student['СНИЛС']]['Приоритет'].values[0]:

                        # Так как значение нового приоритета меньше, убираем абитуриента со старой специальности и добавляем в новую
                        index = students_received[students_received['СНИЛС'] == student['СНИЛС']].index
                        copy_place[students_received.loc[index, 'Категория'].values[0]][students_received.loc[index, 'Специальность'].values[0]] += 1

                        # Удаление информацию об абитуриенте
                        students_received.drop(labels=[index[0]], inplace=True)

                    # Если значение исходного приоритета меньше нового, пропускаем абитуриента
                    else:
                      df_students.drop(labels=[i], inplace=True)
                      continue

                # Добаляем информацию о новом, поступившем абитуриенте
                item = pd.DataFrame([student])
                item["БВИ"] = item["БВИ"].astype("boolean")
                students_received = pd.concat([students_received, item])

                # Обновляем кол-во мест
                copy_place[student['Категория']][student['Специальность']] -= 1

                # Удаляем из общих данных уже поступивших абитуриентов
                df_students.drop(labels=[i], inplace=True)


Для бюджетной основы сначала определяем поступивших абитуриентов на общих основаниях, затем на квотные места,
   оставшиеся места переходят в места 'Основные места', дополняем списки 'Основные места'.

In [14]:
def get_students_received():

    # Определяем поступивших абитуриентов на общих основаниях
    incoming_students('Основные места')
    kvota = ['Особая квота', 'Отдельная квота', 'Целевая квота']

    for category in kvota:
        incoming_students(category)

    # Добавляем оставшиеся места в общие основания
    for category in kvota:
      for speciality in copy_place[category]:
          copy_place['Основные места'][speciality] += copy_place[category][speciality]
          copy_place[category][speciality] = 0

    # Дополняем поступивших абитуриентов на общих основаниях
    incoming_students('Основные места')


In [15]:
get_students_received()

Проверим размерности данных и кол-во уникальных СНИЛСов (для того чтобы убедится, что один абитуриент не попал в наш DataFrame несколько раз).


In [16]:
df_students.shape, students_received.shape

((18, 6), (883, 6))

In [17]:
students_received['СНИЛС'].nunique()

883

Тут можно проверить на какую специальность проходит абитуриент.

In [18]:
snils = input('Введите снилс в формате xxx-xxx-xxx xx или уникальный код: ')
students_received[students_received['СНИЛС'] == snils]

Введите снилс в формате xxx-xxx-xxx xx или уникальный код: 198-688-591 77


,СНИЛС,Специальность,Балл,Приоритет,БВИ,Категория
233,198-688-591 77,04.03.01 Химия,235.0,1.0,False,Целевая квота


Вычислим проходные баллы на специальности.

In [19]:
passing_score = students_received[students_received['БВИ'] == False]\
                .groupby(['Специальность', 'Категория'], as_index=False)\
                .agg({'Балл': min}).sort_values(by='Категория')

In [20]:
passing_score.head(5)


,Специальность,Категория,Балл
0,01.03.02 Прикладная математика и информатика,Основные места,0.0
105,43.03.01 Сервис,Основные места,196.0
62,"26.03.02 Кораблестроение, океанотехника и сист...",Основные места,162.0
106,43.03.02 Туризм,Основные места,194.0
60,24.05.05 Интегрированные системы летательных а...,Основные места,151.0


#Сохранение данных

Сохраняем собранные данные о всех поступивших абитуриентах в Excel таблицу.

In [21]:
students_received.to_excel('students.xlsx')

Сохраним данные о поступивших абитуриентах в Excel таблицу.

In [22]:
writer = pd.ExcelWriter('passing_score_final.xlsx', engine='xlsxwriter')

for category in passing_score['Категория'].unique():
    passing_score[passing_score['Категория'] == category].to_excel(writer, sheet_name=category)

writer.close()